# CUDA y PyCUDA

Google Colab ya viene con CUDA preinstalado, lo cual lo podemos checar con lo siguiente:

In [0]:
b!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


Sin embargo, no tiene PyCUDA, por lo que debemos instalarlo (tardará un par de minutos):

In [0]:
!curl https://colab.chainer.org/install | sh -
!pip install pycuda
!pip install scikit-cuda

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  21944      0 --:--:-- --:--:-- --:--:-- 21944
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


# 1. CUDA

## 1.1 ¿Por qué un GPU?

La principal problemática con un CPU hoy en día, y la razón por la que tenemos que pensar en paralelo es que estamos llegando a un límite de velocidad de las arquitecturas actuales. Esto es debido a que para que tenga una mayor velocidad un procesador, hay que utilizar más potencia en este. Después de cierto punto, se vuelve ineficiente o imposible mantener el procesador lo suficientemente frío como para operar correctamente.

Por otro lado, visualicemos cuantos hilos se pueden ejecutar en paralelo en un CPU. Ejemplo: 



*   Procesador Intel Skylake (2017).
*   Velocidad de 3.5 GHz base.
*   Tiene ocho núcleos físicos.
*   Puede realizar operaciones de vectores de 512 bits, es decir, 16 operaciones simultáneas con números de 32 bits. (AVX)
*   Hyperthreading, básicamente divide el núcleo físico y lo hace ver como dos núcleos lógicos, lo que deja realizar aproximadamente dos hilos al mismo tiempo.
*    Esto nos lleva a un máximo teórico de $
2\cdot16\cdot8 = 256$ hilos paralelos.

Por otro lado:



*   Nvidia GTX 980 (2014).
*   Velocidad de 1.127 GHz base.
*   Posee 2048 unidades de cómputo para sombreado.
*   Esto nos lleva a 2048 hilos en paralelo sólo de unidades de sombreado.

Como podemos notar, incluso varias generaciones atrás, las GPUs podían realizar una mayor cantidad de operaciones concurrentes que una CPU normal, y no están atadas tan fuertemente por las limitaciones de un CPU (correlación directa entre velocidad y nivel de paralelismo, así como velocidad y calor). Hoy en día, esta diferencia se ha incrementado enormemente.

## 1.2 Paradigmas de la programación de GPU

Aunque es claro que se pueden realizar muchas más operaciones concurrentes en un GPU que en un CPU, también es lógico que estas operaciones no pueden resultar tan intensivas como las que le delegaríamos a un CPU. Esto es debido a que los GPU buscan tener más procesadores más pequeños y simples. Asimismo, existen otras limitantes que generan los siguientes tres fundamentos para la programación de un GPU:



*   Como hay muchas unidades de cómputo, se cambia una facilidad de control por más cómputo.
*   Se debe seguir un modelo de programación paralelo.
*   Se optimiza para volumen de cómputo, no latencia.

Cabe mencionar que el modelo de computadora que se utiliza con una combinación de CPU y GPU es denominado heterogéneo. En este modelo, existen dos roles, el de anfitrión y el de dispositivo.

Ambos tienen su propia memoria y capacidades, por lo que el sincronizar y delegar sus trabajos correctamente es vital para poder sacarles provecho.

## 1.3 ¿Qué es CUDA?

CUDA (Compute Unified Device Architecture) es una plataforma de computo paralelo desarrollada por Nvidia para sus GPUs. Esta permite utilizar este hardware para secciones de cómputo altamente paralelizables.

El origen de CUDA se remonta al 2006, cuando se integran conceptos desarrollados previamente para extender C y darle capacidades de paralelismo.

## 1.4 ¿Por qué CUDA?

Por el dominio de mercado de Nvidia, la plataforma CUDA es la más utilizada en aplicaciones diversas como Deep Learning.

Aunque existen competidores, no tienen la misma cantidad de soporte, ni el mismo desempeño en GPUs de Nvidia.

La implementación de CUDA resulta ventajosa debido a que simplifica la programación en un GPU. Esto lo logra debido a dos factores:



*   La facilidad de compartir memoria entre el anfitrión y el dispositivo. Esto debido a que CUDA provee de operaciones de traslado de memoria sencillo entre ambos dispositivos.
*   El uso de las funciones kernel. Un kernel es la sección paralela del código, y CUDA tiene la facilidad de que se implementa como cualquier otra función. Sin embargo, debe de realizarse una asignación cuidadosa del dominio de trabajo de cada kernel individual ya que por defecto un GPU posee una arquitectura SIMD.

## 1.5 Desventajas de CUDA



*   Limitado únicamente a equipos Nvidia.
*   Existe un cuello de botella generado por el bus de datos entre el GPU y el CPU.
*   No es tan popular.


## 1.6 Ventajas de CUDA

*   Un GPU tiene un nivel de paralelización varias magnitudes más alto que un CPU, lo que deriva en una gran capacidad de resolver problemas paralelizables, como por ejemplo, las operaciones tensoriales. En CUDA, la implementación para resolver este tipo de problemas es muy sencilla.
*   Tiene soporte para escalabilidad. Es decir, puedes tener varias GPUs trabajando en paralelo sin muchos problemas.

# 2. PyCUDA

Como ya hemos visto, las GPU Nvidia utilizan CUDA para realizar cálculos en paralelo. Sin embargo, la implementación original de CUDA es en lenguaje C. Si queremos usar CUDA en otros lenguajes, como por ejemplo Python, debemos acercarnos a las implementaciones de CUDA para este mismo. Aquí entra PyCUDA, que es un wrapper para CUDA en Python.
PyCUDA nos da toda la funcionalidad de CUDA implementada en forma de funciones, y nos da una opción de ejecutar el código C de CUDA en el mismo, por lo que resulta muy versátil para la programación con CUDA y Python, es decir. PyCUDA nos da una manera fácil de usar CUDA desde la comodidad de Python.

## 2.1 ¿Por qué PyCUDA?

PyCUDA tiene varias ventajas



* Conveniencia. Abstracciones como pycuda.compiler.SourceModule y pycuda.gpuarray.GPUArray hacen que la programación en CUDA sea aún más conveniente que con el tiempo de ejecución basado en C de Nvidia.
* Comprobación automática de errores. Todos los errores de CUDA se traducen automáticamente en excepciones de Python.
* Velocidad. La capa base de PyCUDA está escrita en C++, por lo que a pesar de que hay un sacrificio en el rendimiento a cambio de la comodidad de Python éste no es dramático.



In [0]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

In [0]:
a_gpu = gpuarray.to_gpu(np.random.randn(4,4).astype(np.float32))

a_doubled = (2*a_gpu).get()


In [0]:
# Convertimos el arreglo a float32 por que las tarjetas gráficas de Nvidia suelen trabajar con solo un grado de precisión
a = np.random.randn(4).astype(np.float32)

# Obtenemos la cantidad de memoria, necesaria para alojar el arreglo de numpy
a_gpu = cuda.mem_alloc(a.nbytes)

############################

# Convertimos el arreglo a float32 por que las tarjetas gráficas de Nvidia suelen trabajar con solo un grado de precisión

b = np.random.randn(4).astype(np.float32)


# Obtenemos la cantidad de memoria, necesaria para alojar el arreglo de numpy
b_gpu = cuda.mem_alloc(b.nbytes)

# Alojamos el arreglo en la localidad de memoria almacenada
cuda.memcpy_htod(b_gpu, b)

############################

# Convertimos el arreglo a float32 por que las tarjetas gráficas de Nvidia suelen trabajar con solo un grado de precisión

c = np.random.randn(4).astype(np.float32)


# Obtenemos la cantidad de memoria, necesaria para alojar el arreglo de numpy
c_gpu = cuda.mem_alloc(c.nbytes)

# Alojamos el arreglo en la localidad de memoria almacenada
cuda.memcpy_htod(c_gpu, c)

mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

mod_a = SourceModule("""
  __global__ void VecAdd(float* A, float* B, float* C)
  {
      int i = threadIdx.x;
      C[i] = A[i] + B[i];
  }
""")


In [0]:
func = mod.get_function("doublify")
func2 = mod_a.get_function("VecAdd")

func(a_gpu, block=(4,4,1))

type(func)

pycuda._driver.Function

In [0]:
func = mod.get_function("doublify")
func2 = mod_a.get_function("VecAdd")

func(a_gpu, block=(4,4,1))

func2(a_gpu, b_gpu, c_gpu, block=(4,4,1))

a_doubled = np.empty_like(a)
b_doubled = np.empty_like(b)
cuda.memcpy_dtoh(a_doubled, a_gpu)
cuda.memcpy_dtoh(b_doubled, c_gpu)


In [0]:
print(b_doubled)
print(a, b, c)

[ 1.1595837 -1.9011077 -2.2535782 15.994909 ]
[ 0.75400907 -0.45637074  1.3098347   0.386344  ] [-1.3792673   0.13927236  1.8463005   0.7482839 ] [-0.07656808  1.310187    0.30812946  1.0180608 ]


### Ejemplo: multiplicación de matrices

In [0]:
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
import numpy as np
import skcuda.linalg
import time

# --- Initializations
skcuda.linalg.init()

A = np.random.random((10,10)).astype(np.float64)
B = np.random.random((10,10)).astype(np.float64)

#t = time.time()

A_gpu = gpuarray.to_gpu(A)
B_gpu = gpuarray.to_gpu(B)

t = time.time()

C_gpu = skcuda.linalg.dot(A_gpu, B_gpu)

print("Tiempo en paralelo: {} s".format(time.time()-t))

t = time.time()

np.dot(A, B)

print("Tiempo secuencial: {} s".format(time.time()-t))

Tiempo en paralelo: 0.006435394287109375 s
Tiempo secuencial: 0.00015783309936523438 s


En lo anterior, vemos que es más ráṕido de manera secuencial, pero eso era con una matriz pequeña, ahora tratemos con una matriz más grande

In [0]:
# --- Initializations
skcuda.linalg.init()

A = np.random.random((5000,5000)).astype(np.float64)
B = np.random.random((5000,5000)).astype(np.float64)

#t = time.time()

A_gpu = gpuarray.to_gpu(A)
B_gpu = gpuarray.to_gpu(B)

t = time.time()

C_gpu = skcuda.linalg.dot(A_gpu, B_gpu)

print("Tiempo en paralelo: {} s".format(time.time()-t))

t = time.time()

np.dot(A, B)

print("Tiempo secuencial: {} s".format(time.time()-t))

Tiempo en paralelo: 0.06437253952026367 s
Tiempo secuencial: 6.9050023555755615 s


### Ejemplo: integrales

In [0]:
import pycuda.autoinit
import pycuda.gpuarray
import numpy as np
import skcuda.integrate
import scipy

In [0]:
dx = 0.001

x = np.arange(0, np.pi, dx, dtype=np.float32) 
y = np.array(np.sin(x**2 + np.cos(x)), dtype=np.float32)

y_gpu = gpuarray.to_gpu(y)

# Empezamos la parte en paralelo
skcuda.integrate.init()

t = time.time()

skcuda.integrate.trapz(y_gpu, dx=dx)

print("Tiempo en paralelo: {} s".format(time.time()-t))

t = time.time()

scipy.trapz(y, x)

print("Tiempo secuencial: {} s".format(time.time()-t))

Tiempo en paralelo: 0.10836362838745117 s
Tiempo secuencial: 0.0008156299591064453 s


In [0]:
dx = 0.001
dy = 0.001

xs = np.arange(0, np.pi, dx, dtype=np.float32) 
ys = np.arange(0, np.pi, dy, dtype=np.float32)

z = np.array([[np.sin(x**2 + np.cos(y)) for x in xs] for y in ys])

#t = time.time()

# Mandamos nuestra matriz a la GPU, pero si la matriz es grande la comunicación toma un tiempo considerable
z_gpu = gpuarray.to_gpu(z)

# Empezamos la parte en paralelo
skcuda.integrate.init()

t = time.time()

skcuda.integrate.trapz2d(z_gpu, dx=dx, dy=dy)

print("Tiempo en paralelo: {} s".format(time.time()-t))

t = time.time()

# Integramos primero sobre x y luego sobre y
scipy.trapz([scipy.trapz(z_x, xs) for z_x in z], ys)

print("Tiempo en secuencial: {} s".format(time.time()-t))

Tiempo en paralelo: 0.10402393341064453 s
Tiempo en secuencial: 0.08076310157775879 s


### Ejemplo: inversa de una matriz

In [0]:
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
import numpy as np
import skcuda.linalg
import time

In [0]:
A = np.random.random((5000,5000)).astype(np.float64)

#t = time.time()

A_gpu = gpuarray.to_gpu(A)

# Empezamos la parte en paralelo
skcuda.linalg.init()

#t = time.time()

skcuda.linalg.inv(A_gpu)

t = time.time()

print("Tiempo en paralelo: {} s".format(time.time()-t))

t = time.time()

np.linalg.inv(A)

print("Tiempo en secuencial: {} s".format(time.time()-t))

Tiempo en paralelo: 5.888938903808594e-05 s
Tiempo en secuencial: 10.10940408706665 s


### Ejemplo: red neuronal

In [0]:
import skcuda.linalg as linalg
import skcuda.misc as misc
import time
import math

In [0]:
class CapaGPU:  
  L=False  
  
  #Funciones de Activación
  def sigmoide(self,z):
    return 1 / (1+math.e**(-1*z))
  def tanh(self,z):
    return (math.e**(z)-math.e**(-1*z))/(math.e**(z)+math.e**(-1*z))

  #Derivadas de las Funciones de Activación
  def sigmoidep(self,z):
    a=self.sigmoide(z)
    return a*(1-a)
  def tanhp (self,z):
    a=self.tanh(z)
    return 1-a**2

    
  def Mul(self,matZ,matY):
    Res=matZ.copy()

    for i in range(matZ.shape[0]):
      for j in range(matZ.shape[-1]):
        Res[i,j]=matZ[i,j]*matY[i,j]
    # linalg.dot(gpuarray.to_gpu(matZ), gpuarray.to_gpu(matY)).get() 
    return Res
  
  def __init__(self,nl_1,nl,act='sigmoide'):
    self.W=np.asmatrix(np.random.randn(nl,nl_1))
    self.b=np.asmatrix(np.random.rand(nl,1))

    if act == 'sigmoide':
      self.g=self.sigmoide
      self.gp=self.sigmoidep
    elif act == 'tanh':
      self.g=self.tanh
      self.gp=self.tanhp
    elif act == 'relu':
      self.g = self.relu
      self.gp=self.relup
    elif act == 'Lrelu':
      self.g = self.Lrelu
      self.gp = self.Lrelup
  
  def Activar(self,matX):
    
    self.Z = self.W*matX + self.b
    # self.W = np.ascontiguousarray(self.W)
    # matX = np.ascontiguousarray(matX)
    # self.Z = misc.multiply(gpuarray.to_gpu(self.W), gpuarray.to_gpu(matX)).get() + self.b
    self.A = self.Z.copy()
    self.Zp = self.Z.copy()
    for i in range(self.Z.shape[0]):
      for j in range(self.Z.shape[-1]):
        self.A[i,j]=self.g(self.Z[i,j])
        self.Zp[i,j]=self.gp(self.Z[i,j])
        
  def Errores(self,matY,matAl_1,Wn1=None,dZn1=None):
    if(self.L):
      self.dA = self.A - matY
    else:
      self.dA = Wn1.T * dZn1
      # self.dA = misc.multiply(gpuarray.to_gpu(Wn1.T), gpuarray.to_gpu(dZn1)).get()
    self.dZ = self.Mul(self.dA,self.Zp)
    self.dW = (1 / self.Z.shape[0] )*self.dZ * matAl_1.T
    self.db = (1/self.Z.shape[0])*np.sum(self.dZ,axis=1)


In [0]:
"""
Definición de la clase Red:
Esta contendrá todas las Capas de la red
Además, tendrá funciones para activar la red,
entrenarla, Backpropagation, Predecir, el discriminador
y una función para separar los datos en los conjuntos de 
entrenamiento, desarrollo y validación.
"""
class RedNeuronalGPU:
 
  def __init__(self):
    self.capas=[]
    self.errores=[]
  def AniadirCapa(self,capa):
    self.capas.append(capa)
    
  def ActRed(self,matX):
    if(len(self.capas)!=0):
      self.capas[0].Activar(matX)
      for i in range(1,len(self.capas)):
       
        self.capas[i].Activar(self.capas[i-1].A)
      self.capas[-1].L=True
    else:
      print("Error! La red no tiene capas")
      
  def BackPropagation(self,matX,matY):
    if len(self.capas)==1:
      self.capas[-1].Errores(matY,matX)
    else:
      self.capas[-1].Errores(matY,self.capas[-2].A)
      for i in reversed(range(1,len(self.capas)-1)):
        self.capas[i].Errores(matY,self.capas[i-1].A,self.capas[i+1].W,self.capas[i+1].dZ)
      self.capas[0].Errores(matY,matX,self.capas[1].W,self.capas[1].dZ)
              
  def Entrenar(self,matX,matY,iteraciones=5000,alpha=0.1,lam=0.0):
    if(len(self.capas)!=0):
      for i in range(iteraciones):
        self.ActRed(matX)
        self.errores.append(1/matX.shape[-1] * np.sum((self.capas[-1].A-matY)*(self.capas[-1].A-matY).T))
        self.BackPropagation(matX,matY)
        for j in range(len(self.capas)):
          self.capas[j].W = self.capas[j].W - alpha*self.capas[j].dW + lam*self.capas[j].W.sum()
          self.capas[j].b = self.capas[j].b - alpha*self.capas[j].db + lam*self.capas[j].b.sum()
    else:
      print("Error! La red no tiene capas")
      
  def Discriminador(self,matZ):
    Res = matZ.copy()
    for i in range(matZ.shape[0]):
      for j in range(matZ.shape[-1]):
        if(i==np.argmax(matZ[:,j])):
          Res[i,j]=1
        else:
          Res[i,j]=0
    return Res  
  
  def Predict(self,matX):
    self.ActRed(matX)
    return self.Discriminador(self.capas[-1].A)
  

In [0]:
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt

# import some data to play with
iris = datasets.load_iris()
X = iris.data  
y = iris.target

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
#Creamos nuestra red 1 
redGPU = RedNeuronalGPU()
redGPU.AniadirCapa(CapaGPU(4,100,'sigmoide'))
redGPU.AniadirCapa(CapaGPU(100,100,'tanh'))
redGPU.AniadirCapa(CapaGPU(100,3,'sigmoide'))

t1 = time.time()

redGPU.Entrenar(X.T, y, 3000)

print("Tiempo en paralelo: {} s".format(time.time()-t1))

#Vemos el error respecto a las iteraciones
plt.plot(np.arange(len(redGPU.errores)),redGPU.errores)

In [0]:
arreglo = gpuarray.to_gpu(redGPU.capas[0].W.T).astype('float32')
arreglo1 = gpuarray.to_gpu(redGPU.capas[1].W.T).astype('float32')
arreglo2 = gpuarray.to_gpu(redGPU.capas[2].W.T).astype('float32')

def sig(z):
  return 1 / (1 + np.exp(-z))

def predecir(x):
  x = gpuarray.to_gpu(x).astype('float32')
  z1 = linalg.dot(x, arreglo).get() + redGPU.capas[0].b.T
  a1 = gpuarray.to_gpu(sig(z1)).astype('float32')
  z2 = linalg.dot(a1, arreglo1).get() + redGPU.capas[1].b.T
  a2 = gpuarray.to_gpu(sig(z2)).astype('float32')
  zL = linalg.dot(a2, arreglo2).get() 
  return sig(zL)

In [0]:



suma = 0
for i in range(150):
  ejemplo = X[i]
  t1 = time.time()

  ale = np.argmax(redGPU.Predict(ejemplo.reshape((len(ejemplo), 1))))
  print("Tiempo en paralelo: {} s".format(time.time()-t1))
  suma += 1 if ale == y[i] else 0



In [0]:


suma = 0
for i in range(150):
  ejemplo = X[i]
  t1 = time.time()
  ale = np.argmax(predecir(ejemplo))
  print("Tiempo en paralelo: {} s".format(time.time()-t1))
  suma += 1 if ale == y[i] else 0



In [0]:
suma

# Referencias útiles



1.   Para checar las funciones de parte de scikit-cuda ver [scikit-cuda](https://scikit-cuda.readthedocs.io/en/latest/reference.html).
2.   Para ver los avances de Nvidia con CUDA checar [Nvidia CUDA Zone](https://developer.nvidia.com/cuda-zone).
3. Algunas aplicaciones en las que se usa CUDA [CUDA Technology](https://www.geforce.com/hardware/technology/cuda).
4. Documentación de CUDA [CUDA Toolkit Documentation](https://docs.nvidia.com/cuda/).
5. Documentación de PyCUDA [PyCUDA Documentation](https://documen.tician.de/pycuda/)
6. Muy breve introduccióón a PyCUDA [PyCUDA Intro](http://www.iitk.ac.in/hpc4se/downloads/PyCUDA-AH.pdf)

